# Filter each stationary frame

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd
import numpy as np
import json
import os

## Constants and such

In [ ]:
# Set background map resolutions
azimuth_resolution = 1
height_resolution = 0.5

In [ ]:
DATA_DIR_ROOT = '../stationary_data'

## First, functions for making background map

In [ ]:
# Create the grid DataFrames
def create_grid_dataframes(azimuth_map, height_map):
    grid_shape = (len(height_map), len(azimuth_map))
    df_distances = pd.DataFrame({key: [[] for _ in range(len(height_map))] \
                                 for key in azimuth_map.keys()}, index=height_map.keys())
    df_intensities = pd.DataFrame({key: [[] for _ in range(len(height_map))] \
                                   for key in azimuth_map.keys()}, index=height_map.keys())
    return df_distances, df_intensities

In [ ]:
# Create mappings for azimuth and height using integers
def create_mappings(azimuth_step=azimuth_resolution, height_step=height_resolution):
    azimuth_range = np.arange(-180, 180 + azimuth_step, azimuth_step)
    height_range = np.arange(-30, 10 + height_step, height_step)
    azimuth_map = {int(az * 10): idx for idx, az in enumerate(azimuth_range)}
    height_map = {int(ht * 10): idx for idx, ht in enumerate(height_range)}
    return azimuth_map, height_map

In [ ]:
# Process file into grid
def process_file_to_grid(file_path, df_distances, df_intensities, azimuth_map, height_map):
    data = np.fromfile(file_path, dtype=np.float32).reshape(-1, 4)
    for x, y, z, intensity in data:
        distance = np.sqrt(x**2 + y**2 + z**2)
        azimuth = np.degrees(np.arctan2(y, x))
        height = np.degrees(np.arctan2(z, np.sqrt(x**2 + y**2)))
        # Convert and scale
        azimuth_idx = int(np.floor((azimuth + 180) / azimuth_resolution) * azimuth_resolution * 10) - 1800
        height_idx = int(np.floor((height + 30) / height_resolution)) - 300
        # Update DataFrames directly using indices
        if azimuth_idx in azimuth_map and height_idx in height_map:
            df_distances.at[height_idx, azimuth_idx].append(distance)
            df_intensities.at[height_idx, azimuth_idx].append(intensity)

In [ ]:
# Function to create the background map DataFrame
# Takes a dataframe of the grid distances and intensity
def create_background_map(df_distances, df_intensities):
    # Initialize a list to store each row (as a dictionary) before creating the DataFrame
    data = []

    # Iterate through each cell in df_distances
    for (height, azimuth), distances in df_distances.stack().items():
        if distances:  # Make sure there are distances recorded in the cell
            # Get the corresponding intensities
            intensities = df_intensities.at[height, azimuth]
            
            # Calculate the median distance and average intensity
            distance = np.max(distances) - np.std(distances)
            average_intensity = np.mean(intensities)

            # Prepare a dictionary for the new row
            new_row = {
                'Azimuth': azimuth / 10,  # Convert back to original scale
                'Height': height / 10,    # Convert back to original scale
                'Distance': distance,
                'Average Intensity': average_intensity
            }
            data.append(new_row)

    # Create the DataFrame from the list of dictionaries
    background_map = pd.DataFrame(data)
    return background_map

In [ ]:
background_map_df = create_background_map(df_distances, df_intensities)

In [ ]:
def generate_background_map(dir):
    pass

## Filtering and saving functions

In [ ]:
def filter_frames(dir, backgroun_map):
    # Create a new folder for the filtered frames in the directory
    # For each frame
    # Filter

In [ ]:
# For each folder in the stationary data directory
p = Path(DATA_DIR_ROOT)

# For each sequence (folder) in the stationary data
for dir in p.iterdir(): 
    if dir.is_dir():
    # Make background map
    background_map = generate_background_map(dir)
    # Filter and save each filtered frame
    filter_frames(dir, background_map)